Para terminar la primera parte del algoritmo vamos a implementar una función que reciba la imagen con ruido, la trate y devuelva varios datos que serán necesarios para la segunda parte.

In [ ]:
import ipynb.fs.defs.CalculaLS as calc
import ipynb.fs.defs.ConsigueUmbrales as conU
import ipynb.fs.defs.Separa as separa
import numpy as np
import cv2

In [ ]:
#Con esta función vamos a dividir los píxeles de la imagen en píxeles con ruido y limpios
def primeraParte(imagen,theta,theta2,d,m):
    
    #Leemos la imagen y hacemos un par de copias necesarias
    im = cv2.imread(imagen, cv2.IMREAD_GRAYSCALE)
    rows, columns = im.shape
    copia = np.zeros((rows,columns), dtype = np.uint8)
    copia2 = np.zeros((rows,columns), dtype = np.uint8)
    
    #Inicializamos las posiciones para las máscara 5x5 que se usa en el artículo
    posiciones = []
    for x in range(-2,3):
        for y in range(-2,3):
            posiciones.append((y,x))
    
    #Recorremos por primera vez los píxeles de la imagen
    ls = []
    for x in range(0, rows):
        for y in range(0, columns):
            vecinos = []
            pixel = ()
            
            #Calculamos las Similaridades Locales(LS) de cada pixel y las añadimos a ls
            for p in posiciones:
                i,j = p
                if p == (0,0):
                    pixel = ((x,y),im[x,y])
                else:
                    v = ((x+i,y+j),im[x+i,y+j])
                    vecinos.append(v)
            calcLS = calc.finalLS(pixel,vecinos)
            ls.append((calcLS,(x,y)))
    
    #Recorremos por segunda vez los píxeles de la imagen
    listaAreas = []
    pixelTipo = []
    for x in range(0, rows):
        for y in range(0, columns):
            
            #Lo primero que vamos a hacer es conseguir las similaridades de los vecinos de cada
            #pixel y sus intensidades de gris
            pixelG = im[x,y]
            vecinosLS = []
            vecinosG = []
            for s in ls:
                sl, pos = s
                for p in posiciones:
                    i,j = p
                    pixel = (x+i,y+j)
                    if p != (0,0):
                        if pos == pixel:
                            vecinosLS.append(sl)
                            vecinosG.append(im[x+i,y+j])
            
            #Con las LS de los vecinos y sus intensidades de gris, podemos calcular en qué tipo de área
            #están, si es plana o compleja. Esto lo vamos a guardar en una lista con el pixel asociado
            #para poder recorrerla luego. Además, con las áreas planas calcularemos los umbrales con
            #los que separaremos los píxeles con ruido de los limpios
            area = separa.Separa(vecinosLS,vecinosG)
            pixelTipo.append((x,y),area)
            if area == "flat":
                listaAreas.append((pixelG,vecinosLS,vecinosG))
            
    #Conseguimos los umbrales
    umPlano, umComp = conU.umbrales(listaAreas,theta,d)
    
    #Recorremos los píxeles y los separamos
    pixRuido = []
    pixLimpios = []
    for tupla in pixelTipo:
        pixel, tipo = tupla
        if tipo == "complex":
            for s in ls:
                sl, pos = s
                if pos == pixel:
                    if sl <= umCom:
                        pixRuido.append(pixel)
                    else:
                        pixLimpios.append(pixel)
        else:
            for s in ls:
                sl, pos = s
                if pos == pixel:
                    if sl <= umPlano:
                        pixRuido.append(pixel)
                    else:
                        pixLimpios.append(pixel)
    
    #Para hacer más robusto el algoritmo vamos ha volver a filtrar aquellos píxeles catalogados
    #como con ruido, ya que si estos píxeles pertenecen a un contorno de la imagen o a una esquina
    #de esta, es posible que se haya catalogado como con ruido cuando en realidad es limpia.
    #Para ello primero tenemos que aplicar el filtro de la mediana y luego el Gaussiano
    copia = cv2.medianBlur(im,5)
    copia2 = cv2.GaussianBlur(copia,5)
    
    #Ahora comprobamos la lista de píxeles con ruido
    for pixel in pixRuido:
        x, y = pixel
        if x == 0 or y == 0 or x == (len(rows)-1) or y == (len(columns)-1):
            valor = im[x,y] - copia2[x,y]
            valorA = abs(valor)
            if valorA <= theta2:
                pixLimpios.append(pixel)
                pixRuido.remove(pixel)
    
    return (pixRuido,pixLimpios)